In [17]:
from datetime import datetime
from helpers import generate_random_payments, convert_to_df, process_ingress_payments
from models.tokens import ERC20Token
from models.dao import Bucket

In [18]:
#build contracts set
token = ERC20Token()
bkt_one = Bucket(token=token, max_volume=100000, name="bkt_01", withdraw_begin=datetime.now())
bkt_two = Bucket(token=token, max_volume=200000, name="bkt_02", withdraw_begin=datetime.now())
bkt_three = Bucket(token=token, max_volume=300000, name="bkt_03", withdraw_begin=datetime.now())
bkt_four = Bucket(token=token, name="bkt_04", withdraw_begin=datetime.now())

bkt_one.set_overflow_bucket(bkt_two)
bkt_one.set_overflow_bucket(bkt_two)
bkt_one.set_overflow_bucket(bkt_two)

In [55]:
# generate random payments
payments = generate_random_payments()
# process each payment and calculate how buckets get filled
processed = process_ingress_payments(payments, token, bkt_one, bkt_two, bkt_three, bkt_four)
columns = processed[0]
dataframe = convert_to_df(processed)

In [56]:
# display the first records of df
dataframe.head()

,income,bkt_01,bkt_02,bkt_03,bkt_04
date,,,,,
2019-01-01 03:36:46,6121,6121,0,0,0
2019-01-05 15:12:32,1714,7835,0,0,0
2019-01-06 10:13:45,7458,15293,0,0,0
2019-01-10 05:15:14,5881,21174,0,0,0
2019-01-13 13:51:39,4116,25290,0,0,0


In [57]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
dframe = dataframe.groupby(lambda x : x.strftime('%y-%m-%d'), sort=False).agg('sum')
dframe['cum_sum'] = dframe.income.cumsum()

from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)
trace1 = {
    'x': dframe.index,
    'y': dframe.income,
    'name': 'Income',
    'type': 'bar',
    'yaxis': 'y'
}
trace2 = {
    'x': dframe.index, 
    'y': dframe.cum_sum,
    'name': 'Total',
    'type': 'scatter',
    'yaxis': 'y2',
    'mode':'lines',
    'line': {
        'shape': 'hvh',
    }
}

layout = {
    'xaxis': {
        'title': 'X axis'
    },
    'yaxis': {
        'title': 'Y Trace',
        'side': 'left',
        'zeroline': True
    },
    'yaxis2': {
        'title': 'Y Total',
        'side': 'right',
        'overlaying': 'y',
    },
    'barmode': 'group',
    'title': 'Payments and raised funds graph', 
}

data = [trace1, trace2]

iplot({'data': data, 'layout': layout})

In [58]:
# aggregation pattern declares which column should be summarized 
# and which should have latest value after
aggpattern = { columns[1] : 'sum' }
for k in columns[2:]:
    aggpattern[k] = 'last'
    
# Aggregate by year+month
grouped_and_aggregated_df = dataframe.groupby(lambda x : x.strftime('%y-%m'), sort=False).agg(aggpattern)
grouped_and_aggregated_df
# Add accumulated summary for investments
grouped_and_aggregated_df['cum_sum'] = grouped_and_aggregated_df.income.cumsum()
grouped_and_aggregated_df

,income,bkt_01,bkt_02,bkt_03,bkt_04,cum_sum
19-01,62820,62820,0,0,0,62820
19-02,50250,100000,13070,0,0,113070
19-03,63034,100000,76104,0,0,176104
19-04,109331,100000,185435,0,0,285435
19-05,50075,100000,200000,35510,0,335510
19-06,97209,100000,200000,132719,0,432719
19-07,109124,100000,200000,241843,0,541843
19-08,89244,100000,200000,300000,31087,631087
19-09,85446,100000,200000,300000,116533,716533
19-10,139000,100000,200000,300000,255533,855533


In [59]:
import pandas as pd
import numpy as np
from random import randint
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

traces = []
for c in columns[1:]:
    trace = {
        'x': grouped_and_aggregated_df.index,
        'y': grouped_and_aggregated_df[c],
        'name': c,
    'type': 'bar',
    'yaxis': 'y'
    }
    traces.append(trace)

trace_cumsum = {
    'x': grouped_and_aggregated_df.index, 
    'y': grouped_and_aggregated_df.cum_sum,
    'name': 'Total',
    'type': 'scatter',
    'yaxis': 'y2',
    'mode':'markers+lines',
    'line': {
        'shape': 'hvh',
    }
}
traces.append(trace_cumsum)

layout = {
    'xaxis': {
        'title': 'X axis'
    },
    'yaxis': {
        'title': 'Y Buckets',
        'side': 'left',
        'zeroline': True
    },
    'yaxis2': {
        'title': 'Y Total invest',
        'side': 'right',
        'overlaying': 'y',
    },
    'barmode': 'group',
    'title': 'Investments and buckets filling', 
}

iplot({'data': traces, 'layout': layout})